In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
book_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/datasets/medical_books"
!cp -R $book_path .

In [ ]:
#langchain library
!pip3 install langchain==0.1.0
#LLM
!pip3 install accelerate==0.25.0
!pip3 install transformers==4.36.2
!pip3 install bitsandbytes==0.41.2
#Text Embedding
!pip3 install sentence-transformers==2.2.2
!pip3 install InstructorEmbedding==1.0.1
#vectorstore
!pip3 install pymupdf==1.23.8
!pip3 install faiss-gpu==1.7.2
!pip3 install faiss-cpu==1.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [ ]:
from langchain import PromptTemplate

# Meidcal GPT Template
prompt_template = """
        Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.
        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.
        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.
        {context}
        Query: {question}
        Answer:
        """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.\n        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.\n        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.\n        {context}\n        Query: {question}\n        Answer:")

In [ ]:
PROMPT.format(
    context = "The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.",
    question = "What is cancer."
)

"Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.\n        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.\n        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.\n        The Asian Institute of Technology (AIT) is an international English-speaking postgraduate institution, focusing on engineering, environment, and management studies. AIT’s rigorous academic, research, and experiential outreach programs prepare graduates for professional success and leadership roles in Asia and beyond.\n        Query: What is cancer.\n        Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

In [ ]:
# multiple pdf

import os
from langchain.document_loaders import PyMuPDFLoader

# Path to the folder containing PDF documents
folder_path = 'medical_books'

# Initialize an empty list to store loaded documents
documents = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    # Check if the file is a PDF
    if file_name.endswith('.pdf'):
        # Construct the full path to the PDF file
        pdf_path = os.path.join(folder_path, file_name)

        # Load the PDF document
        loader = PyMuPDFLoader(pdf_path)
        document = loader.load()

        # Add the loaded documents to the list
        documents.extend(document)

# Now, all_documents contains the loaded documents from all PDFs in the "datasets" folder

In [ ]:
# documents

In [ ]:
len(documents)

6157

In [ ]:
documents[1]

Document(page_content='Medicine\nDavidson’s\nPrinciples and Practice of\n                    PDF  Collected  By:\n        Dr. Nazmul Alam Faruki\n', metadata={'source': 'medical_books/Davidsons_Principles_and_Practice_of_Med.pdf', 'file_path': 'medical_books/Davidsons_Principles_and_Practice_of_Med.pdf', 'page': 1, 'total_pages': 1428, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.2.12', 'producer': '2.3.4 (4.2.13) d', 'creationDate': "D:20220420125723+06'00'", 'modDate': "D:20220420171211+06'00'", 'trapped': ''})

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [ ]:
doc[1]

Document(page_content='Medicine\nDavidson’s\nPrinciples and Practice of\n                    PDF  Collected  By:\n        Dr. Nazmul Alam Faruki', metadata={'source': 'medical_books/Davidsons_Principles_and_Practice_of_Med.pdf', 'file_path': 'medical_books/Davidsons_Principles_and_Practice_of_Med.pdf', 'page': 1, 'total_pages': 1428, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'PDFsam Basic v4.2.12', 'producer': '2.3.4 (4.2.13) d', 'creationDate': "D:20220420125723+06'00'", 'modDate': "D:20220420171211+06'00'", 'trapped': ''})

In [ ]:
len(doc)

45560

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [ ]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [ ]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

create path done


In [ ]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [ ]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)

In [ ]:
#ready to use
retriever = vectordb.as_retriever()

In [ ]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(page_content='\x13\r\x0e\x0f\x0c\r\x13�\n\t\x0f\n\t\n\x13\x0b\t\x1f\x13\x0c\t\x0f\x11\x0b\x0e\n\x1b\x1f\x0b\n\x18�\n\x16\x15\x0c\x1c�\n\x0b\x17\r\x14\x13\r\r\x19�\n\x0e\n\x0b\x14\t\x1b\x14\x13�\n\x12\x0b\r�\n\r\x12\x1b(\x13\x1a�\n\x0b\x1d\x0b\x18�\n\x11\n\x0f\x1f�\n\t\x12\x13�\n\x15\r\x13�\n\x0f\x11�\n\r\x0e\x13\x14\x1b\x0b\x16\x1b)\x13\x1a�\n\t\x13\x14\x12\x0c\x1b\x1e\x15\x13\r�\n\t\x0f\n\x0f\x17\t\x0b\x1b\x0c\x1f\x0b\t\x13\n\x1b\x0b\x16\x11\x0f\n�\n\x14\x15\x16\t\x15\n\x13\x19�\n\r\x15\x14\x12�\n\x0b\r�\n\t\n\x0b\x0c\r\t\n\x0b\x14\x12\x13\x0b\x16�\n\x0b\r\x0e\x1b\n\x0b\t\x1b\x0f\x0c�\n\x0b\x0c\x1a�\n\x17\n\x0f\x0c\x14\x12\x0f\x0b\x16"\x13\x0f\x16\x0b\n�\n\x16\x0b"\x0b\x1c\x13�\n\x1d\x1b\t\x12�\n\x0e\n\x0f\t\x13\x14\t\x13\x1a\n\x17\n\x15\r\x12\r\x0e\x13\x14\x1b\x1f\x13\x0c\r\t\x12\x0b\t�\n\x0b\x16\x16\x0f\x1d�', metadata={'source': 'medical_books/Harrisons_Principles_of_INTERNAL_MEDICIN.pdf', 'file_path': 'medical_books/Harrisons_Principles_of_INTERNAL_MEDICIN.pdf', 'page': 

In [ ]:
retriever.get_relevant_documents("What is Transformers")

[Document(page_content='fibers of the same histochemical type under control of a single anterior horn cell, and thus innervated by the same axon. These can be observed clinically as a small \nmuscle twitch or movement. Electrically a fasciculation is a large-amplitude potential that consists of the simultaneous involuntary depolarization of a group of muscle \nfibers. Although they do not necessarily imply neuromuscular disease and occur in many normal patients, fasciculations can occur in diseases of the motor neuron or \nnerve. Neither clinical nor electrical fasciculations are a feature of muscle disorders.\nEMG ACTIVITY WITH MOVEMENT.', metadata={'source': 'medical_books/cecil-textbook-of-medicine.pdf', 'file_path': 'medical_books/cecil-textbook-of-medicine.pdf', 'page': 2422, 'total_pages': 2572, 'format': 'PDF 1.6', 'title': 'Cecil Textbook of Medicine.pdb', 'author': 'giac', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 7.

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [ ]:
!pip3 install flask-sqlalchemy

In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [ ]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [ ]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios.
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'How may I help you?'})
memory.save_context({"input":'I\'m having a cough and sore throad what should I do?'},{'output': 'Do you have a fever?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: How may I help you?\nHuman: I'm having a cough and sore throad what should I do?\nAI: Do you have a fever?"}

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'How may I help you?'})
memory.save_context({"input":'I\'m having a cough and sore throad what should I do?'},{'output': 'Do you have a fever?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content='How may I help you?'),
  HumanMessage(content="I'm having a cough and sore throad what should I do?"),
  AIMessage(content='Do you have a fever?')]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time.
- it only uses the last K interactions.
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'How may I help you?'})
memory.save_context({"input":'I\'m having a cough and sore throad what should I do?'},{'output': 'Do you have a fever?'})
memory.load_memory_variables({})

{'history': "Human: I'm having a cough and sore throad what should I do?\nAI: Do you have a fever?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available),
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [ ]:
# !rm -rf models

In [ ]:
%mkdir models
%cd ./models
# os.environ["TOKENIZERS_PARALLELISM"] = 'false'
!git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

/content/models
Cloning into 'fastchat-t5-3b-v1.0'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), 8.95 KiB | 381.00 KiB/s, done.
Encountered 1 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin

See: `git lfs help smudge` for more details.


In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [ ]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [ ]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [ ]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': '<pad> What  is  the  difference  between  Machine  Learning  and  Deep  Learning  AI?\n'}

`combine_docs_chain`

In [ ]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.\n        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.\n        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.\n        {context}\n        Query: {question}\n        Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7ee2d8fa5690>)), document_variable_name='context')

In [ ]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.
        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.
        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.
        fibers of the same histochemical type under control of a single anterior horn cell, and thus innervated by the same axon. These can be observed clinically as a small 
muscle twitch or movement. Electrically a fasciculation is a large-amplitude potential that consists of the simultaneous involuntary depolarization of a group of muscle 
fibers. Although they do not necessarily imply 

{'input_documents': [Document(page_content='fibers of the same histochemical type under control of a single anterior horn cell, and thus innervated by the same axon. These can be observed clinically as a small \nmuscle twitch or movement. Electrically a fasciculation is a large-amplitude potential that consists of the simultaneous involuntary depolarization of a group of muscle \nfibers. Although they do not necessarily imply neuromuscular disease and occur in many normal patients, fasciculations can occur in diseases of the motor neuron or \nnerve. Neither clinical nor electrical fasciculations are a feature of muscle disorders.\nEMG ACTIVITY WITH MOVEMENT.', metadata={'source': 'medical_books/cecil-textbook-of-medicine.pdf', 'file_path': 'medical_books/cecil-textbook-of-medicine.pdf', 'page': 2422, 'total_pages': 2572, 'format': 'PDF 1.6', 'title': 'Cecil Textbook of Medicine.pdb', 'author': 'giac', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2.2', 'producer': '

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template="Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.\n        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.\n        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.\n        {context}\n        Query: {question}\n        Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object a

## 5. Chatbot

In [ ]:
# prompt_question = "Who are you by the way?"
# answer = chain({"question":prompt_question})
# answer

In [ ]:
# prompt_question = "What is the Transformers?"
# answer = chain({"question":prompt_question})
# answer

In [ ]:
# prompt_question = "Explain about cancer?"
# answer = chain({"question":prompt_question})

In [ ]:
import cloudpickle

file_path = '/content/models/medical_chatbot.pkl'
with open(file_path, 'wb') as f:
  cloudpickle.dump(chain, f)

In [ ]:
def load_model():
    with open(file_path, 'rb') as f:
        chain = cloudpickle.load(f)
    return chain

model = load_model()

In [ ]:
prompt_question = "I am coughing and have a sore throat. What should I do?"
answer = model({"question":prompt_question})



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Hello! I'm your Medical Assistant, here to provide you with insights and information about health conditions, treatments, and general medical advice.
        Whether you need information on symptoms, advice on health issues, or details about medical procedures, feel free to ask, and I'll do my best to provide you with clear and informative answers.
        From managing common illnesses to understanding complex medical conditions, I'm here to assist you in navigating through your medical queries.
        should be treated with analgesics, fluids, and rest. Persistent cough can be treated with suppressant preparations. Antibiotics are not indicated. Secondary bacterial 
infection complicates mainly influenza virus infection; it should be treated with antibiotics. In pharyngitis, pharyngeal pain or dysphagia should be treated wit

In [ ]:
answer['answer']

'<pad> If  you  are  coughing  and  have  a  sore  throat,  it  is  important  to  seek  medical  attention  as  soon  as  possible.  Here  are  some  steps  you  can  take  to  manage  your  symptoms:\n 1.  Rest:  Taking  a  warm  bath  or  shower  can  help  soothe  your  throat  and  reduce  coughing.\n 2.  Drink  plenty  of  fluids:  Drinking  plenty  of  fluids  can  help  flush  out  any  toxins  in  your  body  and  help  relieve  coughing.\n 3.  Take  analgesics:  Taking  over-the-counter  analgesics  such  as  ibuprofen  or  acetaminophen  can  help  relieve  coughing  and  pain.\n 4.  Avoid  spicy  or  greasy  foods:  Avoid  spicy  or  greasy  foods  as  they  can  irritate  your  throat '

In [ ]:
answer['chat_history']

[]

In [ ]:
prompt_question = "Can I just treat myself at home?"
answer = model({"question":prompt_question})



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='I am coughing and have a sore throat. What should I do?'), AIMessage(content='<pad> If  you  are  coughing  and  have  a  sore  throat,  it  is  important  to  seek  medical  attention  as  soon  as  possible.  Here  are  some  steps  you  can  take  to  manage  your  symptoms:\n 1.  Rest:  Taking  a  warm  bath  or  shower  can  help  soothe  your  throat  and  reduce  coughing.\n 2.  Drink  plenty  of  fluids:  Drinking  plenty  of  fluids  can  help  flush  out  any  toxins  in  your  body  and  help  relieve  coughing.\n 3.  Take  analgesics:  Taking  over-the-counter  analgesics  such  as  ibuprofen  or  acetaminophen  can  help  relieve  coughing  and  pain.\n 4.  Avoid  spicy  or  grea

In [ ]:
answer['answer']

"<pad> Yes,  you  can  treat  a  cough  and  sore  throat  at  home  with  analgesics,  fluids,  and  rest.  However,  it's  important  to  note  that  if  you  have  a  severe  cough  or  sore  throat,  it's  best  to  see  a  doctor  for  proper  diagnosis  and  treatment.  If  you  have  a  cough  that  is  persistent  or  severe,  it's  important  to  see  a  doctor  for  proper  diagnosis  and  treatment.\n"

In [ ]:
# dest = "/content/drive/MyDrive/Colab_Notebooks/NLP/Assignment\ 7/models/"

In [ ]:
file_path

'/content/models/medical_chatbot.pkl'

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/"

In [ ]:
!cp $file_path $save_path